<a href="https://colab.research.google.com/github/benmccloskey/Philadelpia_eagles_analysis/blob/main/Eagles_2023_SB_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [176]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


In [224]:
#Reading in the Dataset 
df = pd.read_csv('/content/drive/MyDrive/eagles_stats_1992_2022.csv')
df.drop('Unnamed: 0', axis = 1, inplace=True)
df.head()

,Year,Wins,Loses,Ties,TOTAL FIRST DOWNS TOTAL 1ST DOWNS,FIRST DOWNS 1ST DOWNS Rushing,FIRST DOWNS 1ST DOWNS Passing,FIRST DOWNS 1ST DOWNS By Penalty,THIRD DOWN CONVERSIONS,FOURTH DOWN CONVERSIONS,...,SACKS,FIELD GOALS,TOUCHDOWNS,TOUCHDOWNS Rushing,TOUCHDOWNS Passing,TOUCHDOWNS Returns,TOUCHDOWNS Defensive,TURNOVER RATIO,Went to Superbowl,Won Superbowl
0,1992,11,5,0,3731,1769,1756,206,1029 / 2658,89 / 177,...,64,16 / 25,43,19,20,1,3,9,0,0
1,1993,8,8,0,3828,1327,2315,186,1165 / 2817,89 / 198,...,42,16 / 23,34,7,23,0,5,1,0,0
2,1994,7,9,0,3694,1308,2114,272,1282 / 3057,74 / 210,...,48,21 / 25,35,14,18,1,2,9,0,0
3,1995,10,6,0,3672,1597,1835,240,1131 / 2947,125 / 212,...,46,22 / 30,32,19,11,1,4,2,0,0
4,1996,10,6,0,4314,1453,2643,218,1120 / 2959,81 / 202,...,39,25 / 29,38,16,19,2,4,-1,0,0


##Preprocessing

In [225]:
#Column Types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 31 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               30 non-null     int64  
 1   Wins                               30 non-null     int64  
 2   Loses                              30 non-null     int64  
 3   Ties                               30 non-null     int64  
 4   TOTAL FIRST DOWNS TOTAL 1ST DOWNS  30 non-null     int64  
 5   FIRST DOWNS 1ST DOWNS Rushing      30 non-null     int64  
 6   FIRST DOWNS 1ST DOWNS Passing      30 non-null     int64  
 7   FIRST DOWNS 1ST DOWNS By Penalty   30 non-null     int64  
 8   THIRD DOWN CONVERSIONS             30 non-null     object 
 9   FOURTH DOWN CONVERSIONS            30 non-null     object 
 10  TOTAL OFFENSIVE YARDS              30 non-null     int64  
 11  OFFENSE Plays                      30 non-null     int64  
 

In [226]:
#Check for null values
df.isna().sum()

Year                                 0
Wins                                 0
Loses                                0
Ties                                 0
TOTAL FIRST DOWNS TOTAL 1ST DOWNS    0
FIRST DOWNS 1ST DOWNS Rushing        0
FIRST DOWNS 1ST DOWNS Passing        0
FIRST DOWNS 1ST DOWNS By Penalty     0
THIRD DOWN CONVERSIONS               0
FOURTH DOWN CONVERSIONS              0
TOTAL OFFENSIVE YARDS                0
OFFENSE Plays                        0
 OFFENSE Average Yards               0
TOTAL RUSHING YARDS                  0
RUSHING Plays                        0
Rushing Average Yards                0
TOTAL PASSING YARDS                  0
PASSING Comp                         0
PASSING Att                          0
PASSING Int                          0
PASSING Avg                          0
SACKS                                0
FIELD GOALS                          0
TOUCHDOWNS                           0
TOUCHDOWNS Rushing                   0
TOUCHDOWNS Passing       

In [227]:
#Summary Statistics
df.describe()

,Year,Wins,Loses,Ties,TOTAL FIRST DOWNS TOTAL 1ST DOWNS,FIRST DOWNS 1ST DOWNS Rushing,FIRST DOWNS 1ST DOWNS Passing,FIRST DOWNS 1ST DOWNS By Penalty,TOTAL OFFENSIVE YARDS,OFFENSE Plays,...,PASSING Avg,SACKS,TOUCHDOWNS,TOUCHDOWNS Rushing,TOUCHDOWNS Passing,TOUCHDOWNS Returns,TOUCHDOWNS Defensive,TURNOVER RATIO,Went to Superbowl,Won Superbowl
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,...,30.000000,30.00000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2006.500000,8.766667,7.166667,0.100000,1232.533333,446.333333,698.633333,87.566667,5501.833333,1041.933333,...,6.840000,42.50000,40.100000,14.166667,23.000000,0.933333,2.866667,0.700000,0.066667,0.033333
std,8.803408,2.661129,2.587548,0.305129,1550.781977,582.624344,884.906599,107.742888,631.408556,48.834649,...,0.803269,10.31487,7.590148,4.720340,6.427312,0.980265,1.676065,9.029263,0.253708,0.182574
min,1992.000000,3.000000,3.000000,0.000000,256.000000,73.000000,146.000000,17.000000,3830.000000,944.000000,...,5.100000,23.00000,17.000000,5.000000,7.000000,0.000000,0.000000,-24.000000,0.000000,0.000000
25%,1999.250000,7.000000,5.250000,0.000000,313.500000,104.250000,183.000000,23.500000,5053.500000,1017.000000,...,6.375000,36.25000,35.250000,11.000000,19.250000,0.000000,2.000000,-6.000000,0.000000,0.000000
50%,2006.500000,9.000000,6.500000,0.000000,338.500000,114.000000,193.000000,30.000000,5613.000000,1048.500000,...,6.800000,41.00000,41.500000,14.500000,22.500000,1.000000,3.000000,2.000000,0.000000,0.000000
75%,2013.750000,10.750000,9.000000,0.000000,2267.750000,799.000000,1283.000000,150.750000,5841.250000,1075.250000,...,7.375000,48.00000,44.750000,16.000000,27.000000,1.750000,4.000000,8.500000,0.000000,0.000000
max,2021.000000,13.000000,13.000000,1.000000,4328.000000,1769.000000,2690.000000,416.000000,6676.000000,1127.000000,...,8.700000,65.00000,52.000000,25.000000,38.000000,4.000000,5.000000,15.000000,1.000000,1.000000


In [228]:
def percentage(dataframe , col_name):
  """This function will transform fractions to percentages 
     within the dataset. 
     
     Parameters
     df (obj): Pandas Dataframe
     col_name (str): Column Name

     Returns:
     The dataframe with an updated column.
     """
  for i,val in enumerate(dataframe[col_name]):
    num1, num2 = val.split('/')
    percent = int(num1)/int(num2)
    dataframe[col_name][i] = percent

  return dataframe

percentage(df, 'THIRD DOWN CONVERSIONS')
percentage(df, 'FOURTH DOWN CONVERSIONS')
percentage(df, 'FIELD GOALS')

df.head()

<ipython-input-228-8ef47a1a850f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_name][i] = percent


,Year,Wins,Loses,Ties,TOTAL FIRST DOWNS TOTAL 1ST DOWNS,FIRST DOWNS 1ST DOWNS Rushing,FIRST DOWNS 1ST DOWNS Passing,FIRST DOWNS 1ST DOWNS By Penalty,THIRD DOWN CONVERSIONS,FOURTH DOWN CONVERSIONS,...,SACKS,FIELD GOALS,TOUCHDOWNS,TOUCHDOWNS Rushing,TOUCHDOWNS Passing,TOUCHDOWNS Returns,TOUCHDOWNS Defensive,TURNOVER RATIO,Went to Superbowl,Won Superbowl
0,1992,11,5,0,3731,1769,1756,206,0.387133,0.502825,...,64,0.64,43,19,20,1,3,9,0,0
1,1993,8,8,0,3828,1327,2315,186,0.413561,0.449495,...,42,0.695652,34,7,23,0,5,1,0,0
2,1994,7,9,0,3694,1308,2114,272,0.419365,0.352381,...,48,0.84,35,14,18,1,2,9,0,0
3,1995,10,6,0,3672,1597,1835,240,0.38378,0.589623,...,46,0.733333,32,19,11,1,4,2,0,0
4,1996,10,6,0,4314,1453,2643,218,0.378506,0.40099,...,39,0.862069,38,16,19,2,4,-1,0,0


In [229]:
#Split the X an Y Variables. We want to predict if the Eagles will go to the superbowl
X = df.drop(['Went to Superbowl', 'Won Superbowl'], axis = 1)
y = df['Went to Superbowl']

print('X Shape: ', X.shape)
print('y Shape: ', y.shape)

X Shape:  (30, 29)
y Shape:  (30,)


In [230]:
#Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8)

In [192]:
df_2 = pd.read_csv('/content/drive/MyDrive/eagles_22-23_seaons.csv')
percentage(df_2, 'THIRD DOWN CONVERSIONS')
percentage(df_2, 'FOURTH DOWN CONVERSIONS')
percentage(df_2, 'FIELD GOALS')
df_2.drop('Unnamed: 0', axis = 1, inplace=True)
df_2.head()

<ipython-input-58-50414f13d77c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col_name][i] = percent


,Year,Wins,Loses,Ties,TOTAL FIRST DOWNS TOTAL 1ST DOWNS,FIRST DOWNS 1ST DOWNS Rushing,FIRST DOWNS 1ST DOWNS Passing,FIRST DOWNS 1ST DOWNS By Penalty,THIRD DOWN CONVERSIONS,FOURTH DOWN CONVERSIONS,...,SACKS,FIELD GOALS,TOUCHDOWNS,TOUCHDOWNS Rushing,TOUCHDOWNS Passing,TOUCHDOWNS Returns,TOUCHDOWNS Defensive,TURNOVER RATIO,Went to Superbowl,Won Superbowl
0,2022,14,3,0,385,161,195,29,0.459459,0.6875,...,44,0.88,59,32,25,0,2,8,0,0


In [206]:
current_season = df_2.drop(['Went to Superbowl', 'Won Superbowl'], axis = 1)
current_season = current_season.to_numpy()

In [208]:
LR = LogisticRegression(
    penalty='l2',
    C=0.1
)
LR.fit(X_train, y_train)
LR.predict_proba(current_season)[:1]

array([[0., 1.]])

In [201]:
xgb = XGBClassifier(
    max_depth=2,
    gamma=2,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)
xgb.fit(X_train, y_train)
xgb.predict_proba(current_season)

array([[0.86779726, 0.13220276]], dtype=float32)

In [202]:
# Synthetic Minority Over Sampling Technique
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy=1, k_neighbors=1, random_state=1)
X_train_over, y_train_over = sm.fit_resample(X_train, y_train)

In [203]:
LR = LogisticRegression(
    penalty='l2',
    C=0.1
)
m.fit(X_train_over, y_train_over)
m.predict_proba(current_season)[:1]

array([[0., 1.]])

In [204]:


m = XGBClassifier(
    max_depth=2,
    gamma=2,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)
m.fit(X_train_over, y_train_over)
m.predict_proba(new_data)

array([[0.09066874, 0.90933126]], dtype=float32)

In [231]:
y = df['Won Superbowl']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=.8)

In [218]:
LR = LogisticRegression(
    penalty='l2',
    C=0.1
)
LR.fit(X_train, y_train)
LR.predict_proba(current_season)[:1]

array([[0., 1.]])

In [219]:
xgb = XGBClassifier(
    max_depth=2,
    gamma=2,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
)
xgb.fit(X_train, y_train)
xgb.predict_proba(current_season)

array([[0.93737185, 0.06262814]], dtype=float32)